In [5]:
!pip install torch_geometric > /dev/null
#pip install rdflib

!pip install torch  > /dev/null
#!pip install scikit-learn > /dev/null


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import  time
import torch
import rdflib
import random
import csv


In [2]:
from src.gnn import *
from src.gnn_utils import *

# 1. Data

This section loads and preprocess the knowledge graph data for entity deduplication.  
The dataset is specified (`cora_dup`, `kgdl_dup`, or `countries_dup`), the `sameAs` relation are defined for evaluation, and the graph structure and metadata are read.  
Dataloaders are then initialized for training, validation, and testing.

In [ ]:

filename = 'cora_dup'
# for kgdl use 'kgdl_dup'
# for countries use 'countries_dup'

to_evaluate = [rdflib.term.URIRef('http://www.w3.org/2002/07/owl#sameAs')]




In [4]:
(data, train_pos_edge_adj, test_pos_edge_adj, _, nodes_dict, relations_dict) = read_data(filename, to_evaluate, same_as_fraction_used_training=1)
same_as_index = relations_dict[rdflib.term.URIRef('http://www.w3.org/2002/07/owl#sameAs')]
relation_ratio = data['train_edge_type'].bincount() / data['train_edge_type'].size(0)
num_nodes = len(nodes_dict)
num_relations = len(relations_dict.keys())
train_loader, test_loader, validation_loader = get_dataloaders(data, to_evaluate, batch_size_train=int(data.train_pos_edge_index.size(1)/4), batch_size_test='all')

Triplets found: 105384


In [5]:
data

Data(edge_index=[2, 105384], edge_type=[105384], test_pos_edge_index=[2, 12915], test_edge_type=[12915], train_pos_edge_index=[2, 92469], train_edge_type=[92469], test_edge_eval_masks=[1, 12915], train_edge_eval_masks=[1, 92469])

# 2. GNN

This section initializes and trains the GNN model for entity deduplication over multiple epochs.  
It reports training and test metrics including Hits@K, Mean Rank (MR), Mean Reciprocal Rank (MRR), and precision/recall/F1 for the `sameAs` relation.  
Training time is also recorded.


In [ ]:
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)
torch.use_deterministic_algorithms(False)

The following hyper-parameters were used

For KGDL use:
    model = GNN(num_nodes=num_nodes, num_relations=num_relations, num_layers=3, hidden_channels=200, learning_rate=1e-2, dropout_prob=0.2, regularization_val=5e-3, same_as_index=same_as_index, relation_ratio=relation_ratio)


For Countries use:
    model = GNN(num_nodes=num_nodes, num_relations=num_relations, num_layers=5, hidden_channels=400, learning_rate=1e-2, dropout_prob=0.2, regularization_val=5e-3, same_as_index=same_as_index, relation_ratio=relation_ratio)

For Cora use:
    model = GNN(num_nodes=num_nodes, num_relations=num_relations, num_layers=3, hidden_channels=300, learning_rate=1e-2, dropout_prob=0.2, regularization_val=5e-3, same_as_index=same_as_index, relation_ratio=relation_ratio)

In [ ]:
st = time.time()
model = GNN(num_nodes=num_nodes, num_relations=num_relations, num_layers=3, hidden_channels=300, learning_rate=1e-2, dropout_prob=0.2, regularization_val=1e-2, same_as_index=same_as_index, relation_ratio=relation_ratio)
epochs = 100

for epoch in range(epochs):
    loss = model.train(train_loader, data['train_pos_edge_index'])
    
    print(f'Epoch: {epoch}, Loss: {loss:.4f}')
    
    hits1_train, hits10_train, mr_train, mrr_train = model.eval(train_loader, train_pos_edge_adj, to_evaluate)
    for i, relation in enumerate(to_evaluate):
        print(f"\t{relation.rsplit('#', 1)[1]} --> Train_Hits@1: {hits1_train[i]:.3f}, Train_Hits@10: {hits10_train[i]:.3f}, Train_MR: {mr_train[i]:.3f}, Train_MRR: {mrr_train[i]:.3f}")
        _, _, _, _, _, _, precision_train, recall_train, f1_train = model.eval_PRF(train_loader, i)
        print(f"\t{relation.rsplit('#', 1)[1]} --> Train_Precision: {precision_train:.3f}, Train_Recall: {recall_train:.3f}, Train_F1: {f1_train:.3f}")
        

    print(f'Epoch: {epoch}, Loss: {loss:.4f}')
    hits1_test, hits10_test, mr_test, mrr_test = model.eval(test_loader, test_pos_edge_adj, to_evaluate)
    for i, relation in enumerate(to_evaluate):
        print(f"\t{relation.rsplit('#', 1)[1]} --> Test_Hits@1: {hits1_test[i]:.3f}, Test_Hits@10: {hits10_test[i]:.3f} Test_MR: {mr_test[i]:.3f}, Test_MRR: {mrr_test[i]:.3f}")
        _, _, _, _, _, _, precision_test, recall_test, f1_test = model.eval_PRF(test_loader, i)
        print(f"\t{relation.rsplit('#', 1)[1]} --> Test_Precision: {precision_test:.3f}, Test_Recall: {recall_test:.3f}, Test_F1: {f1_test:.3f}")
        

et = time.time()
elapsed_time = et - st
print(f'Run time: {elapsed_time:.0f} seconds, {elapsed_time/60:.0f} minutes')

training:   0%|          | 0/5 [00:01<?, ?it/s]


KeyboardInterrupt: 

## Same loop, but writing the results to a file 

In [ ]:
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)
torch.use_deterministic_algorithms(False)

In [ ]:
st = time.time()
model = GNN(num_nodes=num_nodes, num_relations=num_relations, num_layers=3, hidden_channels=300, learning_rate=1e-2, dropout_prob=0.2, regularization_val=1e-2, same_as_index=same_as_index, relation_ratio=relation_ratio)
epochs = 100

for relation in to_evaluate:
    filename_train = f'../R-GCN-deduplication/results/{filename}_train.csv'
    filename_test = f'../R-GCN-deduplication/results/{filename}_test.csv'


with open(filename_train, 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    headerList = ['epoch', 'loss']
    for relation in to_evaluate:
        headerList.extend([f'{relation.rsplit("#", 1)[1]}_train_hits@1',f'{relation.rsplit("#", 1)[1]}_train_hits@10',f'{relation.rsplit("#", 1)[1]}_train_MR',f'{relation.rsplit("#", 1)[1]}_train_MRR', f'{relation.rsplit("#", 1)[1]}_train_precision',f'{relation.rsplit("#", 1)[1]}_train_recall',f'{relation.rsplit("#", 1)[1]}_train_f1'])
    csvwriter.writerow(headerList)
    
with open(filename_test, 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    headerList = ['epoch']
    for relation in to_evaluate:
        headerList.extend([f'{relation.rsplit("#", 1)[1]}_test_hits@1',f'{relation.rsplit("#", 1)[1]}_test_hits@10',f'{relation.rsplit("#", 1)[1]}_test_MR',f'{relation.rsplit("#", 1)[1]}_test_MRR', f'{relation.rsplit("#", 1)[1]}_test_precision',f'{relation.rsplit("#", 1)[1]}_test_recall',f'{relation.rsplit("#", 1)[1]}_test_f1'])
    csvwriter.writerow(headerList)

for epoch in range(epochs):
    print(f'Epoch: {epoch}/{epochs}')
    loss = model.train(train_loader, data['train_pos_edge_index'])
    
    hits1_train, hits10_train, mr_train, mrr_train = model.eval(train_loader, train_pos_edge_adj, to_evaluate)
    row = [epoch, loss]
    for i, relation in enumerate(to_evaluate):
        _, _, _, _, _, _, precision_train, recall_train, f1_train = model.eval_PRF(train_loader, i)
        row.extend([hits1_train[i].item(), hits10_train[i].item(), mr_train[i].item(), mrr_train[i].item(), precision_train, recall_train, f1_train])
        
    
    with open(filename_train, 'a') as csvfile:     
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(row)

    hits1_test, hits10_test, mr_test, mrr_test = model.eval(test_loader, test_pos_edge_adj, to_evaluate)
    row = [epoch]
    for i, relation in enumerate(to_evaluate):
        _, _, _, _, _, _, precision_test, recall_test, f1_test = model.eval_PRF(test_loader, i)
        row.extend([hits1_test[i].item(), hits10_test[i].item(), mr_test[i].item(), mrr_test[i].item(), precision_test, recall_test, f1_test])
    
    with open(filename_test, 'a') as csvfile:     
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(row)

et = time.time()
elapsed_time = et - st
print(f'Run time: {elapsed_time:.0f} seconds, {elapsed_time/60:.0f} minutes')

Epoch: 0/100


training:  60%|██████    | 3/5 [00:11<00:07,  3.73s/it]


KeyboardInterrupt: 